In [ ]:
from pprint import pprint

In [ ]:
# Specify here your MODEL; 
# - e.g. anthropic.claude-3-5-sonnet-20240620-v1:0, meta.llama3-8b-instruct-v1:0, anthropic.claude-3-5-haiku-20241022-v1:0
MODEL_ID="meta.llama3-8b-instruct-v1:0"

#### Dependencias:
> Confirma que hayas instalado los requerimientos.txt, incluyendo langchain*

In [ ]:
from langchain_aws import ChatBedrock

llm = ChatBedrock(
    model_id=MODEL_ID,
    model_kwargs=dict(temperature=0)
)

### Cómo invocar un LLM, a través de LangChain

In [ ]:
messages = [
    (
        "system",
        "Eres un asistente que traduce de Inglés a Español. Traduce el texto del usuario, por favor.",
    ),
    ("human", "I love this Generative AI training."),
]
ai_msg = llm.invoke(messages)
ai_msg

## LangChain Expression Language (LCEL)
> Source [here](https://python.langchain.com/docs/concepts/lcel/)

> Cheatsheets [here](https://python.langchain.com/docs/how_to/lcel_cheatsheet/)

In [ ]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(lambda x: f"Hola, {str(x)}!")
runnable.invoke('mundo')

### Batch a runnable 

In [ ]:
from langchain_core.runnables import RunnableLambda

runnable = RunnableLambda(lambda x: f"Me gusta la serie {str(x)}!")
runnable.batch(['Big Bang Theory','Friends','Black Mirror'])

### Compose runnables

In [ ]:
from langchain_core.runnables import RunnableLambda

runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 4)

chain = runnable1 | runnable2

chain.invoke(2)

### Invoke runnables in parallel

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnableParallel

runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 5)

chain = RunnableParallel(first=runnable1, second=runnable2)

chain.invoke(2)

### Turn any function into a runnable

In [ ]:
from langchain_core.runnables import RunnableLambda

def func(x):
    return x + 5

runnable = RunnableLambda(func)
runnable.invoke(2)

### Merge input and output dicts

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

runnable1 = RunnableLambda(lambda x: x["foo"] + 7)

chain = RunnablePassthrough.assign(bar=runnable1)

chain.invoke({"foo": 10})

# Runnables y Prompts

In [ ]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Function to create the system prompt
def create_system_prompt(_):  # Add an underscore to accept and ignore the input
    return "You are a helpful assistant."

# Function to create the user prompt
def create_user_prompt(input_dict):
    return f"Below you have a series of tasks: \n{input_dict['tasks']}"

# Create RunnableLambda for each prompt
system_prompt_runnable = RunnableLambda(create_system_prompt)
user_prompt_runnable = RunnableLambda(create_user_prompt)

# Create the chain
chain = RunnablePassthrough.assign(
    system_prompt=system_prompt_runnable,
    user_prompt=user_prompt_runnable
)

# Invoke the chain
result = chain.invoke({"tasks": "Task 1: Escribe correo. \nTask 2: Adjunta factura. \nTask 3: Envía correo"})

# Remove the key ´tasks´ from dict, as RunnablePassthrough pass the input as is initially
result.pop("tasks")
pprint(result)

### Prompt Templates
> Source [here](https://python.langchain.com/docs/concepts/prompt_templates/)

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt_template.invoke({"topic": "cats"})

### Chains y Prompt Templates

In [ ]:
from langchain_core.runnables import RunnableLambda

runnable1 = RunnableLambda(lambda x: {"foo": x})
runnable2 = RunnableLambda(lambda x: [x] * 2)

chain = runnable1 | runnable2

chain.invoke(2)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente que traduce de {input_language} a {output_language}.",
        ),
        ("human", "{input}"),
    ]
)

chain = prompt | llm
chain.invoke(
    {
        "input_language": "Inglés",
        "output_language": "Español",
        "input": "I love programming.",
    }
)

In [ ]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Initialize the LLM
llm = ChatBedrock(
    model_id=MODEL_ID,
    model_kwargs=dict(temperature=0)
)

# Create prompt templates for each translation step
spanish_to_german_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that translates from Spanish to German. Please translate the user's text."),
    ("human", "{input}")
])

german_to_english_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant that translates from German to English. Please translate the user's text."),
    ("human", "{input}")
])

# Create the chain
chain = (
    spanish_to_german_prompt 
    | llm 
    | StrOutputParser() 
    | german_to_english_prompt 
    | llm 
    | StrOutputParser()
)

# Invoke the chain
spanish_input = "Me encanta este entrenamiento de IA Generativa."
result = chain.invoke({"input": spanish_input})

print(f"Spanish input: {spanish_input}")
print(f"English output: {result}")

---

In [ ]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda

# Initialize the LLM
llm = ChatBedrock(
    model_id=MODEL_ID,
    model_kwargs=dict(temperature=0)
)

# Create prompt templates for each translation step
spanish_to_german_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce del Español a Alemán. Por favor, traduce el texto del usuario."),
    ("human", "{input}")
])

german_to_english_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce de Alemán a Inglés. Por favor, traduce el texto del usuario."),
    ("human", "{german_text}")
])

# Create the chain
def translate_spanish_to_german(inputs):
    prompt = spanish_to_german_prompt.format_messages(input=inputs["input"])
    german_text = llm.invoke(prompt).content
    return {"input": inputs["input"], "german_text": german_text}

def translate_german_to_english(inputs):
    prompt = german_to_english_prompt.format_messages(german_text=inputs["german_text"])
    english_text = llm.invoke(prompt).content
    return {"input": inputs["input"], "german_text": inputs["german_text"], "english_text": english_text}

chain = RunnableSequence(
    first=RunnableLambda(translate_spanish_to_german),
    last=RunnableLambda(translate_german_to_english)
)

# Invoke the chain
spanish_input = "Me encanta este entrenamiento de IA Generativa."
result = chain.invoke({"input": spanish_input})

print(f"Spanish input: {result['input']}")
print(f"German intermediate: {result['german_text']}")
print(f"English output: {result['english_text']}")

In [ ]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda

# Initialize the LLM
llm = ChatBedrock(
    model_id=MODEL_ID,
    model_kwargs=dict(temperature=0)
)

# Create prompt templates for each translation step
spanish_to_german_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce del Español a Alemán. Por favor, traduce el texto del usuario."),
    ("human", "{input}")
])

german_to_english_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce de Alemán a Inglés. Por favor, traduce el texto del usuario."),
    ("human", "{german_text}")
])

english_to_mandarin_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce de Inglés a Chino Mandarín. Por favor, traduce el texto del usuario."),
    ("human", "{english_text}")
])

# Create the chain
def translate_spanish_to_german(inputs):
    prompt = spanish_to_german_prompt.format_messages(input=inputs["input"])
    german_text = llm.invoke(prompt).content
    return {"input": inputs["input"], "german_text": german_text}

def translate_german_to_english(inputs):
    prompt = german_to_english_prompt.format_messages(german_text=inputs["german_text"])
    english_text = llm.invoke(prompt).content
    return {"input": inputs["input"], "german_text": inputs["german_text"], "english_text": english_text}

def translate_english_to_mandarin(inputs):
    prompt = english_to_mandarin_prompt.format_messages(english_text=inputs["english_text"])
    mandarin_text = llm.invoke(prompt).content
    return {"input": inputs["input"], "german_text": inputs["german_text"], "english_text": inputs["english_text"], "mandarin_text": mandarin_text}


chain = RunnableSequence(
    first=RunnableLambda(translate_spanish_to_german),
    middle=[RunnableLambda(translate_german_to_english)], 
    last=RunnableLambda(translate_english_to_mandarin)
)

# Invoke the chain
spanish_input = "Me encanta este entrenamiento de IA Generativa."
result = chain.invoke({"input": spanish_input})

print(f"Spanish input: {result['input']}")
print(f"German intermediate: {result['german_text']}")
print(f"English intermediate: {result['english_text']}")
print(f"Mandarin output: {result['mandarin_text']}")

In [ ]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda

# Initialize the LLM
llm = ChatBedrock(
    model_id=MODEL_ID,
    model_kwargs=dict(temperature=0)
)

# Create prompt templates for each translation step
spanish_to_german_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce del Español a Alemán. Por favor, traduce el texto del usuario."),
    ("human", "{input}")
])

german_to_english_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce de Alemán a Inglés. Por favor, traduce el texto del usuario."),
    ("human", "{german_text}")
])

english_to_mandarin_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce de Inglés a Chino Mandarín. Por favor, traduce el texto del usuario."),
    ("human", "{english_text}")
])

mandarin_to_portuguese_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce de Chino Mandarín a Portugués. Por favor, traduce el texto del usuario."),
    ("human", "{mandarin_text}")
])

# Create the chain
def translate_spanish_to_german(inputs):
    prompt = spanish_to_german_prompt.format_messages(input=inputs["input"])
    german_text = llm.invoke(prompt).content
    return {"input": inputs["input"], "german_text": german_text}

def translate_german_to_english(inputs):
    prompt = german_to_english_prompt.format_messages(german_text=inputs["german_text"])
    english_text = llm.invoke(prompt).content
    return {"input": inputs["input"], "german_text": inputs["german_text"], "english_text": english_text}

def translate_english_to_mandarin(inputs):
    prompt = english_to_mandarin_prompt.format_messages(english_text=inputs["english_text"])
    mandarin_text = llm.invoke(prompt).content
    return {"input": inputs["input"], "german_text": inputs["german_text"], "english_text": inputs["english_text"], "mandarin_text": mandarin_text}

def translate_mandarin_to_portuguese(inputs):
    prompt = mandarin_to_portuguese_prompt.format_messages(mandarin_text=inputs["mandarin_text"])
    portuguese_text = llm.invoke(prompt).content
    return {"input": inputs["input"], "german_text": inputs["german_text"], "english_text": inputs["english_text"], "mandarin_text": inputs["mandarin_text"], "portuguese_text": portuguese_text}


chain = RunnableSequence(
    first=RunnableLambda(translate_spanish_to_german),
    middle=[
        RunnableLambda(translate_german_to_english),
        RunnableLambda(translate_english_to_mandarin)
        ], 
    last=RunnableLambda(translate_mandarin_to_portuguese)
)

# Invoke the chain
spanish_input = "Me encanta este entrenamiento de IA Generativa."
result = chain.invoke({"input": spanish_input})

print(f"Spanish input: {result['input']}")
print(f"German intermediate: {result['german_text']}")
print(f"English intermediate: {result['english_text']}")
print(f"Mandarin output: {result['mandarin_text']}")
print(f"Portuguese output: {result['portuguese_text']}")

In [ ]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda

# Initialize the LLM
llm = ChatBedrock(
    model_id=MODEL_ID,
    model_kwargs=dict(temperature=0)
)

# Create prompt templates for each translation step
spanish_to_german_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce del Español a Alemán. Por favor, traduce el texto del usuario."),
    ("human", "{input}")
])

german_to_english_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce de Alemán a Inglés. Por favor, traduce el texto del usuario."),
    ("human", "{german_text}")
])

english_to_mandarin_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce de Inglés a Chino Mandarín. Por favor, traduce el texto del usuario."),
    ("human", "{english_text}")
])

mandarin_to_portuguese_prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que traduce de Chino Mandarín a Portugués. Por favor, traduce el texto del usuario."),
    ("human", "{mandarin_text}")
])

# Create the chain
def translate_spanish_to_german(inputs):
    print(f"Recibiendo input en español: '{inputs['input']}'\n")
    print("Traduciendo de español a alemán...")
    prompt = spanish_to_german_prompt.format_messages(input=inputs["input"])
    german_text = llm.invoke(prompt).content
    print(f"Traducción al alemán completada: '{german_text}'\n")
    return {"input": inputs["input"], "german_text": german_text}

def translate_german_to_english(inputs):
    print(f"Recibiendo texto en alemán: '{inputs['german_text']}'")
    print("Traduciendo de alemán a inglés...")
    prompt = german_to_english_prompt.format_messages(german_text=inputs["german_text"])
    english_text = llm.invoke(prompt).content
    print(f"Traducción al inglés completada: '{english_text}'\n")
    return {"input": inputs["input"], "german_text": inputs["german_text"], "english_text": english_text}

def translate_english_to_mandarin(inputs):
    print(f"Recibiendo texto en inglés: '{inputs['english_text']}'")
    print("Traduciendo de inglés a mandarín...")
    prompt = english_to_mandarin_prompt.format_messages(english_text=inputs["english_text"])
    mandarin_text = llm.invoke(prompt).content
    print(f"Traducción al mandarín completada: '{mandarin_text}'\n")
    return {"input": inputs["input"], "german_text": inputs["german_text"], "english_text": inputs["english_text"], "mandarin_text": mandarin_text}

def translate_mandarin_to_portuguese(inputs):
    print(f"Recibiendo texto en mandarín: '{inputs['mandarin_text']}'")
    print("Traduciendo de mandarín a portugués...")
    prompt = mandarin_to_portuguese_prompt.format_messages(mandarin_text=inputs["mandarin_text"])
    portuguese_text = llm.invoke(prompt).content
    print(f"Traducción al portugués completada: '{portuguese_text}'\n")
    return {"input": inputs["input"], "german_text": inputs["german_text"], "english_text": inputs["english_text"], "mandarin_text": inputs["mandarin_text"], "portuguese_text": portuguese_text}

chain = RunnableSequence(
    first=RunnableLambda(translate_spanish_to_german),
    middle=[
        RunnableLambda(translate_german_to_english),
        RunnableLambda(translate_english_to_mandarin)
    ], 
    last=RunnableLambda(translate_mandarin_to_portuguese)
)

# Invoke the chain
spanish_input = "Me encanta este entrenamiento de IA Generativa."
print("Iniciando la cadena de traducción...")
result = chain.invoke({"input": spanish_input})

print("\nResultados finales:")
print(f"Spanish input: {result['input']}")
print(f"German intermediate: {result['german_text']}")
print(f"English intermediate: {result['english_text']}")
print(f"Mandarin output: {result['mandarin_text']}")
print(f"Portuguese output: {result['portuguese_text']}")
